<a href="https://colab.research.google.com/github/LucasArg00/add-automated-tests-off-platform-project/blob/main/Lecture_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GFP Sequence to Function Model

1.   Elemento de lista
2.   Elemento de lista



Code adapted from [David Brookes' CbAS](https://github.com/MauriceR71/CbAS-Restored)

In [ ]:
!git clone https://github.com/igemto-drylab/lec6.git
%cd lec6

fatal: destination path 'lec6' already exists and is not an empty directory.
/content/lec6


In [ ]:
import keras
from keras.layers import Input, Dense, Reshape, Flatten
from keras import layers, initializers
from keras.models import Model, load_model
import keras.backend as K
import numpy as np
from keras.callbacks import EarlyStopping
import pandas as pd
import random

Using TensorFlow backend.


In [ ]:
random.seed(1)

AA = ['a', 'r', 'n', 'd', 'c', 'q', 'e', 'g', 'h', 'i', 'l', 'k', 'm', 'f', 'p', 's', 't', 'w', 'y', 'v', 'x']
AA_upper = []
for m_aa in AA:
    AA_upper.append(m_aa.upper())

AA_IDX = {AA_upper[i]:i for i in range(len(AA_upper))}

def one_hot_encode_aa(aa_str):
    M = len(aa_str)
    aa_arr = np.zeros((M, 21), dtype=int)
    for i in range(M):
        aa_arr[i, AA_IDX[aa_str[i].upper()]] = 1
    return aa_arr

In [ ]:
def get_data(train_size, ignore_stops=True):
    data_df = pd.read_csv("gfp_data.csv")

    idx = data_df.index
    data_df = data_df.loc[idx]

    if ignore_stops:
        idx = data_df.loc[~data_df['aaSequence'].str.contains('!')].index
    data_df = data_df.loc[idx]

    seqs = data_df['aaSequence']

    M = len(seqs[0])
    N = len(seqs)
    X = np.zeros((N, M, 21))
    j = 0
    for i in idx:
        X[j] = one_hot_encode_aa(seqs[i])
        j += 1
    y = np.array(data_df['medianBrightness'][idx])

    # zip and shuffle data
    data_list = list(zip(X, y))
    random.shuffle(data_list)
    X, y = zip(*data_list)

    return np.array(X)[:train_size], np.array(y)[:train_size]

In [ ]:
def build_model(M):
    x = Input(shape=(M, 21,))
    y = Flatten()(x)
    y = Dense(50, activation='elu')(y)
    y = Dense(2)(y)
    model = Model(inputs=x, outputs=y)
    return model

In [ ]:
def neg_log_likelihood(y_true, y_pred):
    y_true = y_true[:, 0]
    mean = y_pred[:, 0]
    variance = K.softplus(y_pred[:, 1]) + 1e-6
    log_variance = K.log(variance)
    return 0.5 * K.mean(log_variance, axis = -1) + 0.5 * K.mean(K.square(y_true - mean) / variance, axis = -1) + 0.5 * K.log(2 * np.pi)

In [ ]:
def train_oracles_helper(X_train, y_train, num_models, batch_size=25):
    for i in range(num_models):
        model = build_model(X_train.shape[1])
        model.compile(optimizer='adam',
                      loss=neg_log_likelihood,
                      )
        early_stop = EarlyStopping(monitor='val_loss',
                                   min_delta=0,
                                   patience=5,
                                   verbose=1)
        # print(model.summary())
        model.fit(X_train, y_train,
                  epochs=250,
                  batch_size=batch_size,
                  validation_split=0.1,
                  callbacks=[early_stop],
                  verbose=2)
        model.save("/content/lec6/oracle_model_%i.h5" % i)

In [ ]:
def train_oracles():
    i = 1
    num_models = 5
    for i in range(num_models):
        X_train, y_train = get_data(train_size=10000)  # data has 58417 seqs

        suffix = '_%i' % num_models
        train_oracles_helper(X_train, y_train, num_models, batch_size=64)

In [ ]:
def make_prediction():
    for i in range(num_models):
        vec = np.array([one_hot_encode_aa(s)])
        model = load_model("/content/lec6/oracle_model_%i.h5" % i)
        predictions.append(model.predict(vec))
    print(predictions)

In [ ]:
train_oracles()

In [ ]:
# seq must have the same number of amino acids as those in the training set
# missing amino acids may be denoted by 'x'
seq = ""

make_predictions(seq)